## 멀티턴을 고려한 쿼리 생성

In [1]:
!pip install langchain langchain_openai langchain_community pypdf faiss-cpu


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import urllib.request
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain import PromptTemplate
from langchain.docstore.document import Document
from typing import List, Dict, Any, Tuple, Optional
from langchain.chat_models import ChatOpenAI
from textwrap import dedent
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.retrievers import BaseRetriever
from langchain.chains import RetrievalQA

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = ""

In [4]:
# PDF 다운로드 및 로드
urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch06/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf", filename="2023_북한인권보고서.pdf")
loader = PyPDFLoader('2023_북한인권보고서.pdf')

이 코드는 원격 저장소에서 PDF 파일을 다운로드하고 LangChain의 PyPDFLoader를 사용하여 문서를 로드하는 과정을 구현합니다.

구체적인 동작과 단계:

1. PDF 파일 다운로드:
  - urllib.request.urlretrieve 함수를 사용하여 원격 URL에서 PDF 파일을 다운로드합니다.
  - URL: GitHub 저장소에 있는 '2023_북한인권보고서.pdf' 파일
  - filename 매개변수를 통해 다운로드된 파일의 로컬 저장 경로를 '2023_북한인권보고서.pdf'로 지정합니다.
  - 이 함수는 파일 다운로드를 완료한 후 로컬 파일 경로를 반환합니다.

2. PDF 문서 로딩:
  - LangChain의 PyPDFLoader 클래스를 사용하여 다운로드된 PDF 파일을 로드합니다.
  - PyPDFLoader는 PDF 파일을 읽고 텍스트를 추출하는데 특화된 로더입니다.
  - 이 시점에서는 파일이 메모리에 완전히 로드되지는 않으며, loader 객체만 생성됩니다.
  - 실제 파일 내용 로딩과 텍스트 추출은 나중에 load() 또는 load_and_split() 메서드 호출 시 이루어집니다.

이 코드는 RAG(Retrieval-Augmented Generation) 시스템의 첫 번째 단계로, 정보 검색의 대상이 될
문서를 준비하는 과정입니다. 이후에는 로드된 PDF 문서를 청크(chunk)로 분할하고, 임베딩을 생성하여
벡터 데이터베이스에 저장하는 단계가 이어집니다.

In [5]:
# 텍스트 분할
# chunk_size=300이면 각 문서 당 최대 길이 300이 넘지 않도록 자르는 모듈.
# chunk_overlap=100은 자를 때 앞의 문서랑 뒤의 문서랑 길이 100정도는 겹치게 해서 자르는 모듈.
doc_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
docs = loader.load_and_split(doc_splitter)

In [6]:
docs[0]

Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2023-07-31T13:50:27+09:00', 'moddate': '2023-07-31T13:57:54+09:00', 'trapped': '/False', 'source': '2023_북한인권보고서.pdf', 'total_pages': 448, 'page': 0, 'page_label': '1'}, page_content='2023\n북한인권보고서\n2023 Report on North Korean Human Rights\n통일부')

In [8]:
docs[1].page_content

'본 보고서는 2017년 이후 북한의 인권실태를 진술한\n북한이탈주민 508명의 증언을 중심으로 작성되었습니다.'

In [10]:
docs[3].page_content

'2023 북한인권보고서\n04\n올해로 유엔의 북한인권조사위원회 출범 10년, 북한인권결의 채\n택 20년이 됩니다. 그동안 우리는 물론 국제사회가 북한인권을 증진\n하기 위해 노력해 왔지만, 휴전선 이북의 북녘 땅은 여전히 최악의 \n인권 사각지대로 남아 있습니다. 우리와 피를 나눈 북한 동포들이 \n최소한의 인간적인 삶을 누릴 수 있도록 책임감을 갖고 보다 실효적\n인 노력을 펼쳐가야만 합니다. \n2016년 제정된 북한인권법에 기반하여 설립된 북한인권기록센\n터는 2017년부터 북한이탈주민을 대상으로 북한의 전반적인 인권'

In [9]:
len(docs[3].page_content)

288

In [11]:
docs[4].page_content

'인 노력을 펼쳐가야만 합니다. \n2016년 제정된 북한인권법에 기반하여 설립된 북한인권기록센\n터는 2017년부터 북한이탈주민을 대상으로 북한의 전반적인 인권\n실태를 심층적으로 조사하였습니다. 또한 파악된 북한의 인권침해 \n사례들을 ‘세계인권선언’과 ‘국제인권조약’의 기준에 따라 분류하였\n습니다. 이번에 발간되는 「북한인권보고서」는 북한의 인권 상황을 \n시민적·정치적 권리, 경제적·사회적·문화적 권리 등 다양한 측면에\n서 입체적으로 조명하였습니다. 아울러, 여성·아동·장애인 등 취약'

In [12]:
# 임베딩 설정
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [13]:
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000026C368CA990>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000026C34923910>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [14]:
# FAISS 벡터스토어 생성 및 저장
faiss_store = FAISS.from_documents(docs, embedding)

In [15]:
# 벡터 DB를 파일로 저장
persist_directory = "/content/DB"
faiss_store.save_local(persist_directory)

# 벡터 DB 저장한 파일을 로드
vectordb = FAISS.load_local(persist_directory, embeddings=embedding, allow_dangerous_deserialization=True)

이 코드는 PDF에서 추출한 텍스트를 처리하고 벡터 데이터베이스를 구축하는 RAG 시스템의 핵심 단계를 구현합니다.

1. 텍스트 분할 (Chunking):
  - RecursiveCharacterTextSplitter를 사용하여 텍스트를 작은 청크로 분할합니다.
  - chunk_size=300: 각 청크의 최대 크기를 300자로 제한합니다.
  - chunk_overlap=100: 인접한 청크 간에 100자의 중복을 허용하여 문맥의 연속성을 유지합니다.
  - loader.load_and_split(): PDF를 로드하고 지정된 splitter를 사용하여 분할합니다.
  - 이 과정에서 북한인권보고서의 내용이 여러 개의 작은 조각으로 나뉘게 됩니다.
  - 적절한 청크 크기 설정은 RAG 시스템의 성능에 중요한 영향을 미칩니다:
    * 너무 작으면 맥락이 손실될 수 있음
    * 너무 크면 관련 정보를 정확히 검색하기 어려울 수 있음

2. 임베딩 설정:
  - OpenAI의 'text-embedding-3-large' 모델을 사용하여 텍스트 임베딩을 생성합니다.
  - 이 모델은 텍스트를 고차원 벡터 공간에 표현하여 의미적으로 유사한 텍스트가
    벡터 공간에서도 가깝게 위치하도록 합니다.
  - 임베딩은 텍스트의 의미적 검색을 가능하게 하는 핵심 요소입니다.

3. FAISS 벡터스토어 생성:
  - FAISS(Facebook AI Similarity Search)는 효율적인 벡터 유사도 검색을 위한 라이브러리입니다.
  - from_documents() 메서드를 통해 분할된 문서와 임베딩 모델을 사용하여 벡터스토어를 생성합니다.
  - 이 과정에서 각 텍스트 청크는 임베딩 벡터로 변환되어 FAISS 인덱스에 저장됩니다.

4. 벡터스토어 영구 저장:
  - persist_directory를 '/content/DB'로 설정하여 벡터스토어를 로컬에 저장합니다.
  - save_local() 메서드를 사용하여 FAISS 인덱스를 디스크에 저장합니다.
  - 이렇게 하면 프로그램을 재시작해도 처음부터 임베딩을 다시 계산할 필요 없이
    저장된 인덱스를 로드할 수 있습니다.

5. 벡터 DB 로드:
  - load_local() 메서드를 사용하여 저장된 FAISS 인덱스를 메모리에 로드합니다.
  - allow_dangerous_deserialization=True: 직렬화된 Python 객체를 안전하지 않더라도
    로드할 수 있게 허용합니다(주의: 신뢰할 수 있는 소스에서만 사용해야 함).
  - 로드된 vectordb 객체는 이후 유사도 검색에 사용됩니다.

In [16]:
# 쿼리 재생성 클래스 정의
class QueryRewriter:
    def __init__(self, model_name="gpt-4o", temperature=0):
        self.llm = ChatOpenAI(temperature=temperature, model_name=model_name, max_tokens=1000)
        self.template = """
        당신은 사용자의 이전 대화와 현재 질문을 기반으로 검색 쿼리를 재생성하는 전문가입니다.

        # 지침
        1. 이전 대화 맥락과 현재 질문을 고려하여 더 정확한 검색 쿼리를 생성하세요.
        2. 현재 질문이 간략하거나 이전 대화의 맥락을 참조하는 경우, 이전 대화를 고려하여 완전한 쿼리를 구성하세요.
        3. 이전 대화가 없거나 관련이 없는 경우 현재 질문만 사용하세요.
        4. 반드시 명확하고 검색에 적합한 쿼리를 생성하세요.
        5. 출력은 재생성된 쿼리 문자열만 포함해야 합니다. 추가 설명이나 주석은 포함하지 마세요.

        # 입력
        이전 대화 질문: {prev_query}
        이전 대화 답변: {prev_response}
        현재 질문: {current_query}

        # 출력
        재생성된 쿼리:
        """
        self.prompt = PromptTemplate(
            input_variables=["prev_query", "prev_response", "current_query"],
            template=self.template
        )

    def rewrite_query(self, prev_query: str, prev_response: str, current_query: str) -> str:
        # 이전 대화가 없는 경우, LLM을 호출하지 말고 현재 검색어를 반환
        if not prev_query or prev_query.strip() == "":
            return current_query

        # 쿼리 재생성. 프롬프트 템플릿과 LLM 객체를 연결
        chain = self.prompt | self.llm

        # 연결된 체인 객체를 호출
        rewritten_query = chain.invoke({
            "prev_query": prev_query,
            "prev_response": prev_response,
            "current_query": current_query
        }).content.strip()

        return rewritten_query

QueryRewriter 클래스는 대화형 검색 시스템에서 대화 맥락을 고려한 쿼리 재생성을 담당합니다. 이 클래스의 핵심 목적은 사용자의 모호하거나 짧은 후속 질문을 이전 대화 맥락을 활용해 더 구체적이고 명확한 검색 쿼리로 변환하는 것입니다.
클래스는 초기화 시 ChatOpenAI 모델을 temperature 0으로 설정하여 결정론적인 응답을 보장합니다. 프롬프트 템플릿은 LLM에게 5가지 주요 지침을 제공합니다: 맥락 고려, 불완전한 질문 완성, 관련 없는 맥락 무시, 검색에 적합한 명확한 쿼리 생성, 그리고 추가 설명 없이 쿼리만 출력하는 것입니다.


rewrite_query 메서드는 세 가지 매개변수를 처리합니다:

- prev_query: 이전 대화에서 사용자가 한 질문
- prev_response: 이전 질문에 대한 시스템의 응답  
- current_query: 현재 사용자의 질문

이 메서드는 두 가지 주요 경우를 처리합니다:
첫째, 이전 대화가 없는 경우(첫 번째 질문인 경우), 예를 들어 사용자가 "서울의 관광지 추천해줘"라고 처음 물었을 때는 현재 질문을 그대로 반환합니다. 이는 맥락이 없으므로 쿼리 재생성이 필요 없기 때문입니다.
둘째, 이전 대화가 있는 경우, LLM을 사용하여 현재 질문을 이전 맥락과 결합합니다. 예를 들어:

- 이전 질문: "한국의 관광지 추천해줘"
- 이전 응답: "한국의 유명한 관광지로는 서울의 경복궁, 부산의 해운대 등이 있습니다..."
- 현재 질문: "부산은?"
- 재생성된 쿼리: "부산의 관광지 추천해줘"

또 다른 예시로:

- 이전 질문: "어르신들을 위한 서울 여행지 추천해줘"
- 이전 응답: "어르신들에게 적합한 서울 여행지로는..."
- 현재 질문: "전주는?"
- 재생성된 쿼리: "어르신들을 위한 전주 여행지 추천해줘"

이렇게 재생성된 쿼리는 벡터 검색 엔진에 전달되어, 단순히 "부산은?" 또는 "전주는?"이라는 모호한 질문으로 검색했을 때보다 훨씬 관련성 높은 문서를 검색할 수 있게 합니다.
프롬프트의 마지막 부분은 LLM이 쿼리 문자열만 반환하도록 지시하여, 파싱이 쉽고 검색 엔진에 바로 사용할 수 있는 형태로 출력을 제한합니다. 이는 추가 후처리 없이 결과를 직접 활용할 수 있게 해줍니다.

In [17]:
# 커스텀 검색기 정의
class ConversationalRetriever(BaseRetriever):
    vectorstore: Any = None
    query_rewriter: Any = None
    prev_query: str = ""
    prev_response: str = ""

    def __init__(self, vectorstore, query_rewriter, **kwargs):
        super().__init__()
        self.vectorstore = vectorstore
        self.query_rewriter = query_rewriter
        self.prev_query = kwargs.get("prev_query", "")
        self.prev_response = kwargs.get("prev_response", "")

    def update_conversation(self, query: str, response: str):
        self.prev_query = query
        self.prev_response = response

    def get_relevant_documents(self, query: str, num_docs=4) -> List[Document]:
        # 쿼리 재생성
        # 이전 질문, 이전 답변, 현재 질문을 바탕으로 새 쿼리를 만드는 작업
        rewritten_query = self.query_rewriter.rewrite_query(
            self.prev_query,
            self.prev_response,
            query
        )

        print(f"원래 쿼리: {query}")
        print(f"재생성된 쿼리: {rewritten_query}")

        # 벡터 검색 수행
        docs = self.vectorstore.similarity_search(rewritten_query, k=num_docs)
        return docs

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        raise NotImplementedError("Async retrieval not implemented")

C:\Users\user\AppData\Local\Temp\ipykernel_7572\3942955254.py:2: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class ConversationalRetriever(BaseRetriever):
C:\Users\user\AppData\Local\Temp\ipykernel_7572\3942955254.py:2: DeprecationWarning: Retrievers must implement abstract `_aget_relevant_documents` method instead of `aget_relevant_documents`
  class ConversationalRetriever(BaseRetriever):


ConversationalRetriever 클래스는 LangChain의 BaseRetriever를 상속받아 대화 맥락을 고려한 문서 검색기를 구현합니다. 이 클래스는 이전 대화를 추적하고, QueryRewriter를 활용하여 쿼리를 재생성한 후 벡터 검색을 수행합니다.
클래스 속성으로는 벡터 저장소(vectorstore), 쿼리 재생성기(query_rewriter), 이전 질문(prev_query), 이전 응답(prev_response)이 있습니다. 이 속성들은 대화 맥락을 유지하는 데 필요한 정보를 저장합니다.
초기화 메서드(init)는 벡터 저장소와 쿼리 재생성기를 필수 인자로 받고, 선택적으로 이전 대화 정보를 받을 수 있습니다. 이전 대화 정보가 제공되지 않으면 빈 문자열로 초기화됩니다.
- update_conversation 메서드는 가장 최근의 질문과 응답을 저장하여 대화 맥락을 업데이트합니다. 이 메서드는 대화가 진행될 때마다 호출되어 최신 맥락을 유지합니다.
- get_relevant_documents 메서드는 BaseRetriever의 핵심 메서드를 구현한 것으로, 사용자 쿼리에 대해 관련 문서를 검색합니다. 이 메서드는 다음과 같은 과정으로 작동합니다:

query_rewriter를 사용하여 이전 대화 맥락(prev_query, prev_response)과 현재 쿼리를 고려한 새로운 쿼리를 생성합니다.
원래 쿼리와 재생성된 쿼리를 콘솔에 출력하여 디버깅을 용이하게 합니다.
벡터 저장소의 similarity_search 메서드를 호출하여 재생성된 쿼리에 가장 관련성 높은 문서를 검색합니다.
검색된 문서 목록을 반환합니다.

예를 들어, 사용자가 이전에 "한국의 전통음식 추천해줘"라고 물은 후 "서울에서는 뭐가 유명해?"라고 질문했다면:

- 원래 쿼리: "서울에서는 뭐가 유명해?"
- 재생성된 쿼리: "서울에서 유명한 한국 전통음식 추천해줘"
- 이 재생성된 쿼리로 벡터 DB를 검색하여 더 정확한 결과를 얻습니다.

클래스는 또한 비동기 검색을 위한 aget_relevant_documents 메서드의 인터페이스를 제공하지만, 현재 구현되지 않았고 NotImplementedError를 발생시킵니다. 필요한 경우 추후에 비동기 검색 기능을 추가할 수 있습니다.

In [18]:
# 메인 RAG 클래스 정의
class ConversationalRAG:
    def __init__(self, vectorstore, model_name="gpt-4o", temperature=0.2):
        # 쿼리 재생성기 선언
        self.query_rewriter = QueryRewriter(model_name=model_name)

        # 쿼리 재생성을 포함하여 검색해주는 검색기
        self.retriever = ConversationalRetriever(vectorstore=vectorstore, query_rewriter=self.query_rewriter)

        # LLM 선언
        self.llm = ChatOpenAI(temperature=temperature, model_name=model_name)

        # 이 모든걸 연결해주는 Retrieval
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff", # RAG 프롬프트 디폴트값
            retriever=self.retriever,
            return_source_documents=True
        )

    def query(self, current_query: str) -> Dict:
        # RAG 검색 및 응답 생성
        result = self.qa_chain.invoke(current_query)

        # 대화 기록 업데이트
        self.retriever.update_conversation(current_query, result["result"])

        return result

ConversationalRAG 클래스는 대화형 검색 기반 질의응답(RAG) 시스템의 핵심 구성 요소로, 모든 컴포넌트를 통합하고 사용자 질의에 대한 응답을 생성합니다.
초기화 메서드(init)에서는 시스템의 주요 구성 요소들을 설정합니다:

- QueryRewriter 인스턴스를 생성하여 대화 맥락을 고려한 쿼리 재생성을 담당하게 합니다.
- ConversationalRetriever 인스턴스를 생성하고, 이전에 만든 query_rewriter와 함께 vectorstore(문서 벡터 데이터베이스)를 연결합니다.
- ChatOpenAI 모델을 지정된 temperature로 초기화하여 최종 응답 생성에 사용합니다.
- LangChain의 RetrievalQA 체인을 구성하여 검색과 응답 생성을 연결합니다. "stuff" 체인 타입은 모든 검색된 문서를 하나의 컨텍스트로 결합하는 가장 단순한 방식입니다.

query 메서드는 사용자의 현재 질문을 받아 전체 RAG 과정을 실행합니다:

1. qa_chain을 호출하여 현재 질문을 처리합니다. 이 과정에서 내부적으로 다음이 수행됩니다:

  - retriever가 쿼리를 재생성하고 관련 문서를 검색
  - 검색된 문서와 질문을 LLM에 전달하여 응답 생성


2. 생성된 응답과 현재 질문을 retriever의 대화 기록에 업데이트합니다. 이는 다음 질문에서 대화 맥락을 유지하기 위한 중요한 단계입니다.
3. 최종 결과를 반환합니다. 이 결과에는 생성된 응답(result["result"])과 검색에 사용된 소스 문서(result["source_documents"])가 포함됩니다.

예를 들어, 사용자가 "19년 말 평양시 소재 기업소에서 달마다 배급받은 음식"이라고 질문한 후 "그럼 일반 주민들은?"이라고 물었을 때:

1. 첫 번째 질문은 그대로 처리되어 관련 정보를 반환
2. 두 번째 질문은 이전 맥락을 고려하여 "19년 말 평양시 소재 일반 주민들이 달마다 배급받은 음식은 무엇인가?"와 같은 형태로 재구성
3. 재구성된 쿼리로 검색을 수행하고 응답 생성
4. 새로운 질문과 응답을 대화 기록에 저장

이 클래스는 RAG 파이프라인의 진입점 역할을 하며, 사용자는 단순히 query 메서드를 호출하는 것만으로 복잡한 대화형 검색 및 응답 생성 기능을 손쉽게 활용할 수 있습니다.

In [19]:
# 사용 예시
conversational_rag = ConversationalRAG(vectordb)

C:\Users\user\AppData\Local\Temp\ipykernel_7572\1227733256.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(temperature=temperature, model_name=model_name, max_tokens=1000)


In [20]:
# 예시 1: 첫 번째 질문 (이전 대화 없음)
query1 = "19년 말 평양시 소재 기업소에서 달마다 배급받은 음식"
result1 = conversational_rag.query(query1)
print(f"\n질문: {query1}")
print(f"답변: {result1['result']}")

원래 쿼리: 19년 말 평양시 소재 기업소에서 달마다 배급받은 음식
재생성된 쿼리: 19년 말 평양시 소재 기업소에서 달마다 배급받은 음식

질문: 19년 말 평양시 소재 기업소에서 달마다 배급받은 음식
답변: 2019년 평양시에서 기업소 운전원으로 일하였던 노동자는 매월 쌀, 설탕, 기름, 야채, 돼지고기 등을 배급받았다고 합니다. 또한, 중앙당 산하의 기업소에서는 매월 쌀 6㎏, 기름 5ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 정도를 받았다는 증언이 있습니다.


In [21]:
# 예시 2: 후속 질문
query2 = "그럼 일반 주민들은?"
result2 = conversational_rag.query(query2)
print(f"\n질문: {query2}")
print(f"답변: {result2['result']}")

원래 쿼리: 그럼 일반 주민들은?
재생성된 쿼리: 2019년 말 평양시 일반 주민들이 매월 배급받은 음식 종류와 양

질문: 그럼 일반 주민들은?
답변: 제공된 정보에 따르면, 일반 주민들은 식량 배급을 받는 데 있어서 기관이나 기업소에 따라 상당한 차이가 있는 것으로 보입니다. 일부 지역에서는 배급이 비교적 원활하게 이루어졌지만, 다른 지역에서는 규정에 미치지 못하는 적은 양의 식량을 받는 경우도 많았습니다. 예를 들어, 평양시에서는 기업소에 따라 매달 3~5일분 정도의 옥수수를 배급받았다는 진술이 있으며, 일부 지역에서는 1년에 한 번 배급이 이루어졌다는 사례도 있습니다. 따라서 일반 주민들은 식량 배급에 있어 불규칙성과 부족함을 경험하고 있는 것으로 보입니다.


In [22]:
# 예시 3: 후속 질문
query3 = "그럼 고위 간부들은?"
result3 = conversational_rag.query(query3)
print(f"\n질문: {query3}")
print(f"답변: {result3['result']}")

원래 쿼리: 그럼 고위 간부들은?
재생성된 쿼리: 고위 간부들의 식량 배급 상황 및 일반 주민들과의 차이점

질문: 그럼 고위 간부들은?
답변: 죄송하지만, 제공된 정보에서는 북한의 고위 간부들에 대한 식량 배급이나 생활 조건에 대한 구체적인 내용이 언급되어 있지 않습니다.


In [23]:
# 예시 3: 주제 전환 질문
query3 = "북한의 교육 시스템은 어떻게 되나요?"
result3 = conversational_rag.query(query3)
print(f"\n질문: {query3}")
print(f"답변: {result3['result']}")

원래 쿼리: 북한의 교육 시스템은 어떻게 되나요?
재생성된 쿼리: 북한의 교육 시스템 구조와 특징

질문: 북한의 교육 시스템은 어떻게 되나요?
답변: 북한의 교육 시스템은 2012년에 개편되어 전반적 의무교육 체제를 갖추고 있습니다. 이 체제는 유치원 1년, 소학교 5년, 초급중학교 3년, 고급중학교 3년으로 구성되어 있습니다. 이전에는 초급중학교와 고급중학교를 통합하여 중학교 6년 과정을 운영하였습니다. 북한은 사회주의헌법에 따라 교육을 받을 권리를 보장하고 있으며, 교육법, 보통교육법, 고등교육법 등을 통해 이를 실현하고 있습니다. 그러나 일반교육보다 정치사상교육을 중시하며, 교과과정에 군사훈련을 포함하여 학생들이 의무적으로 참석하도록 하고 있습니다.


## 멀티턴 + 멀티쿼리까지 고려한 쿼리 생성

In [24]:
import os
import urllib.request
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain import PromptTemplate
from langchain.docstore.document import Document
from typing import List, Dict, Any, Tuple, Optional
from langchain.chat_models import ChatOpenAI
from textwrap import dedent
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.retrievers import BaseRetriever
from langchain.chains import RetrievalQA
import json

In [ ]:
# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = "여러분들의 Key 값"

# PDF 다운로드 및 로드
urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch06/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf", filename="2023_북한인권보고서.pdf")
loader = PyPDFLoader('2023_북한인권보고서.pdf')

In [ ]:
# 텍스트 분할
doc_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
docs = loader.load_and_split(doc_splitter)

# 임베딩 설정
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

# FAISS 벡터스토어 생성 및 저장
faiss_store = FAISS.from_documents(docs, embedding)
persist_directory = "/content/DB"
faiss_store.save_local(persist_directory)

# 벡터 DB 로드
vectordb = FAISS.load_local(persist_directory, embeddings=embedding, allow_dangerous_deserialization=True)

In [25]:
# 멀티 쿼리 재생성 모델 정의
class MultiQueryGenerator(BaseModel):
    queries: List[str] = Field(description="사용자 질문에서 추출한 검색 쿼리 목록")

In [26]:
# 수정이 필요한 AdvancedQueryRewriter 클래스의 프롬프트 템플릿 부분만 수정

class AdvancedQueryRewriter:
    def __init__(self, model_name="gpt-4o", temperature=0):
        self.llm = ChatOpenAI(temperature=temperature, model_name=model_name, max_tokens=1000)
        self.parser = JsonOutputParser(pydantic_object=MultiQueryGenerator)

        # 프롬프트 템플릿 수정 - 명확한 형식 지침 추가
        self.template = """
        당신은 사용자의 이전 대화와 현재 질문을 기반으로 검색 쿼리를 재생성하는 전문가입니다.

        # 지침
        1. 이전 대화 맥락과 현재 질문을 고려하여 더 정확한 검색 쿼리를 생성하세요.
        2. 현재 질문이 간략하거나 이전 대화의 맥락을 참조하는 경우, 이전 대화를 고려하여 완전한 쿼리를 구성하세요.
        3. 이전 대화가 없거나 관련이 없는 경우 현재 질문만 사용하세요.
        4. 현재 질문에 여러 개의 질의가 포함되어 있다면, 각각을 별도의 쿼리로 분리하세요.
        5. 각 쿼리는 독립적으로 벡터 DB에서 검색될 수 있도록 완전하고 명확해야 합니다.
        6. 마크다운 형식으로 작성하지 마십시오.

        # 입력
        이전 대화 질문: {prev_query}
        이전 대화 답변: {prev_response}
        현재 질문: {current_query}

        # 출력 형식
        반드시 아래의 JSON 형식으로 출력하세요:
        {{
          "queries": ["쿼리1", "쿼리2", ...]
        }}

        예시:
        현재 질문이 "서울은? 그리고 맛집은?"이고 이전 대화가 "한국의 관광지 추천해줘"라면,
        {{
          "queries": ["서울의 관광지 추천해줘", "서울의 맛집 추천해줘"]
        }}

        단일 쿼리인 경우:
        {{
          "queries": ["한국의 관광지 추천해줘"]
        }}

        {format_instructions}
        """

        self.prompt = PromptTemplate(
            input_variables=["prev_query", "prev_response", "current_query", "format_instructions"],
            template=self.template
        )

    # rewrite_query 메서드는 그대로 유지
    def rewrite_query(self, prev_query: str, prev_response: str, current_query: str) -> List[str]:
        # 이전 대화가 없는 경우 간단히 처리
        if not prev_query or prev_query.strip() == "":
            # 단일 질문인 경우
            if "?" in current_query and current_query.count("?") == 1:
                return [current_query]
            else:
                # 복잡한 질문은 LLM으로 처리
                pass

        try:
            # 프롬프트 준비
            format_instructions = self.parser.get_format_instructions()

            # 체인 실행
            formatted_prompt = self.prompt.format(
                prev_query=prev_query,
                prev_response=prev_response,
                current_query=current_query,
                format_instructions=format_instructions
            )

            # 직접 LLM 호출
            llm_response = self.llm.invoke(formatted_prompt)

            # JSON 직접 파싱 (파서 사용 대신)
            try:
                import json
                response_json = json.loads(llm_response.content)
                if "queries" in response_json and isinstance(response_json["queries"], list):
                    return response_json["queries"]
                else:
                    print(f"응답에 'queries' 리스트가 없습니다: {response_json}")
                    return [current_query]
            except json.JSONDecodeError as json_err:
                print(f"JSON 파싱 오류: {str(json_err)}")
                print(f"LLM 원본 응답: {llm_response.content}")
                return [current_query]
        except Exception as e:
            print(f"쿼리 재생성 중 오류 발생: {str(e)}")
            print(f"LLM 원본 응답: {llm_response.content}")
            # 오류 발생 시 원래 쿼리 사용
            return [current_query]

AdvancedQueryRewriter 클래스는 대화형 RAG(Retrieval-Augmented Generation) 시스템에서 멀티턴 대화 맥락과 멀티 쿼리 처리를 모두 지원하는 고급 쿼리 재생성 컴포넌트입니다.

주요 기능과 작동 방식:

1. 멀티턴 대화 처리:
  - 이전 대화의 질문과 응답을 기억하여 현재 질문의 맥락을 파악합니다.
  - 예: 이전 질문이 "한국의 관광지 추천해줘"이고 현재 질문이 "서울은?"이라면,
    "서울의 관광지 추천해줘"로 확장합니다.

2. 멀티 쿼리 생성:
  - 하나의 사용자 질문에서 여러 개의 독립적인 질의를 식별하고 분리합니다.
  - 예: "전주는? 그리고 거기서 살 관광 상품도 있을까?"라는 질문은
    ["전주의 관광지 추천해줘", "전주에서 살 수 있는 관광 상품 추천해줘"]와 같은
    두 개의 별도 쿼리로 분리됩니다.

3. 구조화된 JSON 출력:
  - 모든 생성된 쿼리를 {"queries": ["쿼리1", "쿼리2", ...]} 형식의 JSON으로 반환합니다.
  - 이 형식은 후속 처리와 멀티 쿼리 검색을 용이하게 합니다.

4. 강화된 오류 처리:
  - JSON 파싱 오류, LLM 응답 오류 등 다양한 예외 상황에 대응합니다.
  - 오류 발생 시 원래 사용자 쿼리를 단일 요소 리스트로 반환하여 시스템 안정성을 보장합니다.
  - 디버깅을 위해 상세한 오류 메시지와 원본 LLM 응답을 출력합니다.

구현 세부 사항:
- ChatOpenAI 모델을 temperature=0으로 설정하여 일관된 결과를 보장합니다.
- 프롬프트 템플릿은 LLM에게 정확한 지침과 출력 형식을 제공합니다.
- 특히 '현재 질문에 여러 개의 질의가 포함되어 있다면, 각각을 별도의 쿼리로 분리하세요.'라는
 지침은 멀티 쿼리 생성의 핵심입니다.
- rewrite_query 메서드는 이전 버전과 달리 문자열 리스트를 반환합니다.

사용 예시:
1. 단일 질의 확장:
  이전 질문: "어르신들을 위한 서울 여행지 추천해줘"
  현재 질문: "전주는?"
  결과: ["어르신들을 위한 전주 여행지 추천해줘"]

2. 복합 질의 분리:
  이전 질문: "어르신들을 위한 서울 여행지 추천해줘"
  현재 질문: "전주는? 그리고 거기서 살 위한 관광 상품도 있을까?"
  결과: ["어르신들을 위한 전주 여행지 추천해줘", "전주 여행가서 살 어르신들을 위한 전주 관광 상품도 있을까?"]

이 클래스는 RAG 시스템의 검색 정확도를 크게 향상시키며, 특히 사용자가 복잡하거나 다중 질문을 할 때
더 관련성 높은 정보를 제공할 수 있게 합니다.

In [27]:
# 멀티 쿼리 검색 클래스 정의
class MultiQueryRetriever:
    def __init__(self, vectorstore, query_rewriter, **kwargs):
        self.vectorstore = vectorstore
        self.query_rewriter = query_rewriter
        self.prev_query = kwargs.get("prev_query", "")
        self.prev_response = kwargs.get("prev_response", "")

    def update_conversation(self, query: str, response: str):
        self.prev_query = query
        self.prev_response = response

    def retrieve(self, query: str, num_docs=3) -> List[Document]:
        # 쿼리 재생성 (여러 개의 쿼리로)
        rewritten_queries = self.query_rewriter.rewrite_query(
            self.prev_query,
            self.prev_response,
            query
        )

        print(f"원래 쿼리: {query}")
        print(f"재생성된 쿼리들: {rewritten_queries}")

        # 모든 쿼리에 대해 검색 수행 및 결과 병합
        all_docs = []
        seen_contents = set()  # 중복 제거를 위한 집합

        # 다수의 쿼리가 주어졌을 때 1개씩 검색
        for idx, rewritten_query in enumerate(rewritten_queries):
            print(f"쿼리 {idx+1}: {rewritten_query}")

            # 벡터 검색 수행
            docs = self.vectorstore.similarity_search(rewritten_query, k=num_docs)

            # 중복 제거하며 문서 추가
            for doc in docs:
                if doc.page_content not in seen_contents:
                    seen_contents.add(doc.page_content)
                    # 메타데이터에 쿼리 정보 추가
                    if not hasattr(doc, 'metadata') or doc.metadata is None:
                        doc.metadata = {}
                    doc.metadata['query'] = rewritten_query
                    all_docs.append(doc)

        print(f"총 {len(all_docs)}개의 고유 문서를 검색했습니다.")
        return all_docs

MultiQueryRetriever 클래스는 멀티턴 대화 맥락과 멀티 쿼리 처리를 모두 지원하는 고급 문서 검색 컴포넌트입니다. 이 클래스는 이전의 ConversationalRetriever를 확장하여 여러 개의 쿼리를 동시에 처리할 수 있는 기능을 추가했습니다.

주요 특징과 동작 방식:

1. 초기화 및 상태 관리:
  - vectorstore: 문서 검색을 위한 벡터 데이터베이스를 참조합니다.
  - query_rewriter: 대화 맥락과 멀티 쿼리를 처리하는 AdvancedQueryRewriter 인스턴스를 사용합니다.
  - prev_query와 prev_response: 이전 대화의 질문과 응답을 저장하여 대화 맥락을 유지합니다.
  - BaseRetriever를 상속하지 않고 독립적인 클래스로 구현되었습니다.

2. 대화 맥락 업데이트:
  - update_conversation 메서드는 현재 질문과 응답을 저장하여 다음 질문에서 참조할 수 있게 합니다.
  - 이는 '그럼 서울은?'과 같은 맥락 의존적 질문을 처리할 때 필수적입니다.

3. 멀티 쿼리 검색 프로세스:
  - retrieve 메서드는 query_rewriter를 통해 단일 사용자 질문에서 여러 검색 쿼리를 생성합니다.
  - 각 쿼리에 대해 별도로 벡터 검색을 수행하고 결과를 병합합니다.
  - 예: "전주는? 거기 맛집도 알려줘"라는 질문은 ["전주 관광지 추천", "전주 맛집 추천"]과 같은
    여러 쿼리로 확장되어 각각 검색됩니다.

4. 검색 결과 최적화:
  - 중복 제거: seen_contents 집합을 사용하여 여러 쿼리에서 중복되는 문서를 제거합니다.
  - 이는 검색 결과의 다양성을 보장하고 중복 정보를 제거하여 사용자 경험을 향상시킵니다.
  - 메타데이터 강화: 각 문서에 어떤 쿼리에서 검색되었는지 정보를 메타데이터로 추가합니다.
    이 정보는 후속 처리나 디버깅에 유용할 수 있습니다.

5. 로깅 및 디버깅:
  - 검색 과정의 각 단계를 콘솔에 출력하여 쿼리 재생성과 검색 결과를 모니터링할 수 있습니다.
  - 원래 쿼리, 재생성된 쿼리들, 각 개별 쿼리, 최종 검색된 문서 수 등이 출력됩니다.

실제 동작 예시:
- 사용자 질문: "북한의 식량 상황은? 그리고 인권 문제는 어떻게 되나요?"
- 재생성된 쿼리들: ["북한의 식량 상황", "북한의 인권 문제 현황"]
- 각 쿼리별로 벡터 DB 검색 수행 (각각 3개씩 문서 검색)
- 중복 제거 후 총 5-6개 정도의 고유 문서 반환 (일부 문서는 두 쿼리에서 모두 검색될 수 있음)

이 클래스는 복잡한 사용자 질문에 대해 더 포괄적이고 다양한 정보를 제공할 수 있게 하며,
대화형 RAG 시스템의 검색 기능을 크게 향상시킵니다. 특히 여러 주제나 질문을 한 번에 물어보는
사용자 상호작용 패턴에 효과적으로 대응할 수 있습니다.

In [28]:
# RAG 응답 생성 클래스 정의
class AdvancedConversationalRAG:
    def __init__(self, vectorstore, model_name="gpt-4o", temperature=0.2):
        # 쿼리 재생성기
        self.query_rewriter = AdvancedQueryRewriter(model_name=model_name)
        # 재생성된 쿼리를 바탕으로 각각의 검색어를 따로 검색한 뒤에 검색 결과를 취합하는 멀티 쿼리 리트리버
        self.retriever = MultiQueryRetriever(vectorstore=vectorstore, query_rewriter=self.query_rewriter)
        # 답변을 해줄 답변 용도의 LLM 객체
        self.llm = ChatOpenAI(temperature=temperature, model_name=model_name)

        # 응답 생성을 위한 프롬프트 템플릿
        self.response_template = """
        당신은 사용자 질문에 대한 정보를 제공하는 도우미입니다.

        사용자 질문: {query}

        다음 정보를 참고하여 사용자 질문에 답변하세요:
        {context}

        참고 사항:
        1. 사용자 질문에 여러 개의 질의가 포함되어 있다면, 각각에 대해 명확하게 답변하세요.
        2. 제공된 정보만을 사용하여 답변하세요. 정보가 부족하면 솔직히 모른다고 답변하세요.
        3. 답변은 한국어로 제공하세요.
        4. 제공된 정보의 원본 출처가 있다면 인용해주세요.

        답변:
        """

        self.response_prompt = PromptTemplate(
            input_variables=["query", "context"],
            template=self.response_template
        )

    def query(self, current_query: str) -> Dict:
        # 관련 문서 검색
        docs = self.retriever.retrieve(current_query)

        # 문서 내용을 컨텍스트로 변환
        context = "\n\n".join([f"문서 {i+1}:\n{doc.page_content}" for i, doc in enumerate(docs)])

        # 응답 생성
        formatted_prompt = self.response_prompt.format(
            query=current_query,
            context=context
        )

        # 직접 LLM 호출
        result = self.llm.invoke(formatted_prompt)
        response = result.content

        # 대화 기록 업데이트
        self.retriever.update_conversation(current_query, response)

        # 결과 반환
        return {
            "query": current_query,
            "result": response,
            "source_documents": docs
        }

AdvancedConversationalRAG 클래스는 멀티턴 대화와 멀티 쿼리를 모두 지원하는 완전한 RAG(Retrieval-Augmented Generation) 시스템을 구현합니다. 이 클래스는 모든 구성 요소를 통합하고 사용자 질의에 대한 최종 응답을 생성하는 핵심 컴포넌트입니다.

주요 특징과 작동 방식:

1. 구성 요소 초기화:
  - AdvancedQueryRewriter: 멀티턴 맥락과 멀티 쿼리를 처리하는 고급 쿼리 재생성기를 생성합니다.
  - MultiQueryRetriever: 여러 쿼리를 동시에 처리하고 결과를 병합하는 검색기를 생성합니다.
  - ChatOpenAI: 최종 응답 생성에 사용되는 언어 모델을 초기화합니다.
  - 이전 ConversationalRAG와 달리 RetrievalQA 체인을 사용하지 않고, 직접 프롬프트와 LLM을 제어합니다.

2. 응답 생성 프롬프트:
  - 사용자 질문과 검색된 컨텍스트를 포함하는 상세한 프롬프트 템플릿을 정의합니다.
  - 특히 '사용자 질문에 여러 개의 질의가 포함되어 있다면, 각각에 대해 명확하게 답변하세요'라는
    지침은 멀티 쿼리 처리를 위해 중요합니다.
  - RAG 시스템의 핵심 원칙인 '제공된 정보만을 사용하여 답변하세요'를 강조합니다.

3. 쿼리 처리 과정:
  - query 메서드는 사용자의 현재 질문을 입력으로 받습니다.
  - MultiQueryRetriever를 통해 멀티턴 맥락을 고려하고 필요시 여러 쿼리로 분리하여 관련 문서를 검색합니다.
  - 예: "북한의 식량 사정은? 인권 문제는 어떤가?"라는 질문은 두 개의 쿼리로 나뉘어 각각 검색됩니다.
  - 검색된 모든 문서를 번호가 매겨진 형식으로 컨텍스트에 통합합니다.

4. 응답 생성 및 대화 업데이트:
  - 사용자 질문과 검색된 컨텍스트를 포함한 프롬프트를 LLM에 전달합니다.
  - LLM은 모든 하위 질의에 답변하는 통합된 응답을 생성합니다.
  - 생성된 응답과 현재 질문을 retriever의 대화 기록에 저장하여 다음 턴에서 맥락을 유지합니다.

5. 결과 반환:
  - 원본 질의, 생성된 응답, 검색에 사용된 소스 문서를 포함하는 딕셔너리를 반환합니다.
  - 이를 통해 클라이언트는 답변뿐만 아니라 해당 답변의 근거가 된 정보도 확인할 수 있습니다.

실제 동작 예시:
- 사용자 입력: "서울은? 그리고 맛집은?"
- 이전 대화: "한국의 관광지 추천해줘"
- 재생성된 쿼리: ["서울의 관광지 추천해줘", "서울의 맛집 추천해줘"]
- 각 쿼리로 문서 검색 및 결과 병합
- 통합된 응답 생성: "서울의 주요 관광지로는... 서울에서 꼭 가봐야 할 맛집으로는..."

이 클래스는 대화형 RAG 시스템의 가장 상위 레벨 컴포넌트로, 사용자는 단순히 query 메서드를 호출하는
것만으로 멀티턴 대화와 복잡한 질의를 처리할 수 있습니다. 사용자 경험 측면에서 볼 때, 여러 관련 질문을
한 번에 물어볼 수 있고, 이전 대화 맥락이 자동으로 고려되므로 더 자연스럽고 효율적인 정보 검색이 가능합니다.

In [29]:
conversational_rag = AdvancedConversationalRAG(vectordb)

In [30]:
# 예시 1: 첫 번째 질문 (이전 대화 없음, 멀티 쿼리도 아님)
query1 = "19년 말 평양시 소재 기업소에서 달마다 배급받은 음식"
print("\n" + "="*50)
print(f"질문 1: {query1}")
result1 = conversational_rag.query(query1)
print(f"답변 1:\n{result1['result']}")


질문 1: 19년 말 평양시 소재 기업소에서 달마다 배급받은 음식
원래 쿼리: 19년 말 평양시 소재 기업소에서 달마다 배급받은 음식
재생성된 쿼리들: ['2019년 말 평양시 소재 기업소에서 매달 배급받은 음식 종류']
쿼리 1: 2019년 말 평양시 소재 기업소에서 매달 배급받은 음식 종류
총 3개의 고유 문서를 검색했습니다.
답변 1:
2019년 말 평양시 소재 기업소에서 매월 배급받은 음식에 대한 정보는 다음과 같습니다:

- 쌀: 약 6kg
- 기름: 5ℓ
- 설탕: 2kg
- 맛내기: 2봉지
- 돼지고기: 2kg
- 닭고기: 1마리

이 정보는 2019년 평양시에서 기업소 운전원으로 일하였던 노동자의 증언에 기반한 것입니다. 이 노동자는 매월 이러한 식량을 배급받아 식량이 부족하지 않았다고 증언하였습니다. (출처: 문서 1)


In [31]:
# 예시 2: 후속 질문 (멀티턴 o, 멀티쿼리x)
query2 = "그럼 일반 주민들은?"
result2 = conversational_rag.query(query2)
print(f"\n질문: {query2}")
print(f"답변: {result2['result']}")

원래 쿼리: 그럼 일반 주민들은?
재생성된 쿼리들: ['2019년 말 평양시 일반 주민들이 매월 배급받은 음식']
쿼리 1: 2019년 말 평양시 일반 주민들이 매월 배급받은 음식
총 3개의 고유 문서를 검색했습니다.

질문: 그럼 일반 주민들은?
답변: 일반 주민들에 대한 식량 배급 상황은 기관이나 지역에 따라 다소 차이가 있는 것으로 보입니다. 평양시의 경우, 식량 배급이 비교적 원활하게 이루어지고 있으며, 대학생들에게는 하루에 쌀 500g이 15일에 한 번씩 배급된다는 진술이 있습니다. 또한, 양강도 보천군에서는 감자 수확철에 세대 당 인원을 기준으로 감자를 배급한 사례가 있습니다. 그러나 문서에서 일반 주민들 전체에 대한 구체적인 배급 상황은 명시되어 있지 않으므로, 더 자세한 정보는 제공되지 않았습니다.


In [32]:
# 예시 3: 주제 전환 질문 (멀티턴o, 멀티쿼리o)
query3 = "혹시 교육 시스템은 어떻게 되나요? 그리고 주민들이 좋아하는 음식이 있나요?"
result3 = conversational_rag.query(query3)
print(f"\n질문: {query3}")
print(f"답변: {result3['result']}")

원래 쿼리: 혹시 교육 시스템은 어떻게 되나요? 그리고 주민들이 좋아하는 음식이 있나요?
재생성된 쿼리들: ['북한의 교육 시스템', '북한 주민들이 좋아하는 음식']
쿼리 1: 북한의 교육 시스템
쿼리 2: 북한 주민들이 좋아하는 음식
총 6개의 고유 문서를 검색했습니다.

질문: 혹시 교육 시스템은 어떻게 되나요? 그리고 주민들이 좋아하는 음식이 있나요?
답변: 북한의 교육 시스템에 대해 말씀드리겠습니다. 북한은 2012년에 전반적 의무교육 체제를 개편하여 유치원 1년, 소학교 5년, 초급중학교 3년, 고급중학교 3년으로 구성된 학제를 운영하고 있습니다. 이는 이전의 중학교 6년 과정에서 변경된 것입니다. 또한, 북한은 사회주의헌법에 따라 모든 공민에게 교육을 받을 권리를 보장하고 있으며, 이를 위해 다양한 교육 관련 법률을 제정하고 있습니다. (출처: 문서 1, 문서 3)

북한 주민들이 좋아하는 음식에 대한 정보는 제공된 문서에서 명확히 언급되지 않았습니다. 다만, 명절에는 떡, 돼지고기, 수산물 등이 배급된 사례가 있으며, 군부대에서는 쌀밥이 급식되었다는 사례가 있습니다. (출처: 문서 4) 이러한 정보로 볼 때, 떡이나 돼지고기, 수산물 등이 북한 주민들이 선호하는 음식일 가능성이 있습니다. 그러나 주민들이 특별히 좋아하는 음식에 대한 구체적인 정보는 제공된 문서에 포함되어 있지 않습니다.
